# Dependencias

In [1]:
%pip install opencv-contrib-python
%pip install plotly
import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import scipy.signal as signal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Definiciones

In [2]:
# Carga el video
nombre_video = 'yoyo.mp4'
cap = cv2.VideoCapture('Videos/' + nombre_video) # Local
if not cap.isOpened():
  cap = cv2.VideoCapture('/content/' + nombre_video) # Colab


# Medidas tomadas de pixeles y metros
medida_metros = 0.10

if nombre_video == 'yoyoCL.mp4':
  medida_pixeles = 110 # Cantidad de pixeles en 0.1 metros
  # Definir rectangulos a trackear segun el primer frame
  bboxCentro = (465, 70, 73, 78)
  bboxBorde = (511, 83, 20, 21)
elif nombre_video == 'yoyo.mp4':
  medida_pixeles = 145
  # Definir rectangulos a trackear segun el primer frame
  bboxCentro = (534, 155, 11, 13)
  bboxBorde = (557, 174, 15, 16)
const_XY = medida_metros/medida_pixeles

# Variables para guardar las posiciones y tiempos
positionsCentro = []
positionsCentro_m = []
positionsBorde = []
positionsBorde_m = []
fps = cap.get(cv2.CAP_PROP_FPS)  # Obtiene los FPS del video
frame_duration = 1.0 / fps  # Duración de cada frame en segundos # Si hay error acá, revisar video
frame_count = 0

# Trackeo

In [3]:
def calcularPosiciones(frame, bbox, positions, positions_m, number):
    # Calcular centro
    (x, y, w, h) = [int(v) for v in bbox]
    center_x = x + w // 2
    center_y = y + h // 2
    # Dibujar bounding box
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Dibuja centro
    cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

    # Guarda posiciones
    current_time = frame_count * frame_duration
    positions.append((current_time, center_x, cap.get(cv2.CAP_PROP_FRAME_HEIGHT) - center_y))
    positions_m.append((current_time, center_x * const_XY, (cap.get(cv2.CAP_PROP_FRAME_HEIGHT) - center_y) * const_XY))

In [4]:
# Inicializar bounding boxes
bbox1 = bboxCentro
bbox2 = bboxBorde
# Crear trackers
trackerCentro = cv2.TrackerCSRT_create()
trackerBorde = cv2.TrackerCSRT_create()
# Leer primer frame
ret, frame = cap.read()
if not ret:
    print("No se pudo leer el video.")
    cap.release()
    cv2.destroyAllWindows()
# Inicializar trackers
ok1 = trackerCentro.init(frame, bbox1) # Si hay error acá, compilar definiciones
ok2 = trackerBorde.init(frame, bbox2)

while True:
    # Lee el siguiente frame
    ret, frame = cap.read()

    if not ret:
        break

    # Actualiza ambos trackers
    ok1, bbox1 = trackerCentro.update(frame)
    ok2, bbox2 = trackerBorde.update(frame)
    if ok1 and ok2:
        calcularPosiciones(frame, bbox1, positionsCentro, positionsCentro_m, 1)
        calcularPosiciones(frame, bbox2, positionsBorde, positionsBorde_m, 2)

    # Muestra el frame con el seguimiento SOLO LOCAL, NO COLAB
    #display_frame = cv2.resize(frame, (400, 1000))
    #cv2.imshow('Object Tracking', display_frame)

    # Rompe el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Incrementa el contador de frames
    frame_count += 1

# Libera la captura y cierra las ventanas
cap.release()
cv2.destroyAllWindows()

# Datos

In [5]:
#Función para pasar de coordenadas cartesianas a polares

def cartesian_to_polar(x_coords, y_coords):
    '''
    Esta función devuelve una lista con el radio r y otra lista con el ángulo theta medido desde la horizontal para las coordenadas polares a
    partir de las coordenadas cartesianas.
    x: lista con los valores de x
    y: lista con los valores de y
    '''
    r_coords = [] # Inicializo una lista vacía para los valores de la coordenada r
    theta_coords = [] # Inicializo una lista vacía para los valores del ángulo medido desde el eje +x
    for x, y in zip(x_coords, y_coords):
        r = np.sqrt(x**2 + y**2)
        theta = np.arctan2(y, x)
        r_coords.append(r)
        theta_coords.append(theta)

    return r_coords, theta_coords

In [6]:
def crear_dataframe(positions): 
    # Crea el DataFrame y aplica filtros
    df = pd.DataFrame(positions, columns=['Time (sec)', 'X', 'Y'])
    window_size = 26
    polynomial_order = 2
    df['X'] = signal.savgol_filter(df['X'], window_size, polynomial_order)
    df['Y'] = signal.savgol_filter(df['Y'], window_size, polynomial_order)

    # Calcular coordenadas polares
    df['R'], df['Theta'] = cartesian_to_polar(df['X'], df['Y'])  # Agregar coordenadas polares

    # Continuar con el resto del cálculo
    df['Delta_Time'] = df['Time (sec)'].diff().fillna(0)
    df['Delta_X'] = df['X'].diff().fillna(0)
    df['Delta_Y'] = df['Y'].diff().fillna(0)

    # Velocidades
    df['Speed_X'] = df['Delta_X'] / df['Delta_Time']
    df['Speed_Y'] = df['Delta_Y'] / df['Delta_Time']
    df['Speed_X'] = df['Speed_X'].replace([np.inf, -np.inf], 0)
    df['Speed_Y'] = df['Speed_Y'].replace([np.inf, -np.inf], 0)
    df['Speed'] = np.sqrt(df['Speed_X']**2 + df['Speed_Y']**2)
    df['Speed'] = df['Speed'].replace([np.inf, -np.inf], 0)

    # Savitzky-Golay filters para suavizar las velocidades
    df['Speed_X'] = signal.savgol_filter(df['Speed_X'], window_size, polynomial_order)
    df['Speed_Y'] = signal.savgol_filter(df['Speed_Y'], window_size, polynomial_order)
    df['Speed'] = signal.savgol_filter(df['Speed'], window_size, polynomial_order)

    # Calcular aceleraciones
    df['Acceleration_X'] = df['Speed_X'].diff() / df['Delta_Time']
    df['Acceleration_Y'] = df['Speed_Y'].diff() / df['Delta_Time']
    df['Acceleration_X'] = df['Acceleration_X'].replace([np.inf, -np.inf], 0)
    df['Acceleration_Y'] = df['Acceleration_Y'].replace([np.inf, -np.inf], 0)
    df['Acceleration'] = np.sqrt(df['Acceleration_X']**2 + df['Acceleration_Y']**2)
    df['Acceleration'] = df['Acceleration'].replace([np.inf, -np.inf], 0)

    # Cálculo de la velocidad angular
    df['Delta_Theta'] = df['Theta'].diff().fillna(0)  # Cambio en el ángulo
    df['Angular_Velocity'] = df['Delta_Theta'] / df['Delta_Time']  # Velocidad angular en radianes por segundo
    df['Angular_Velocity'] = df['Angular_Velocity'].replace([np.inf, -np.inf], 0)  # Eliminar infinitos
    df['Angular_Velocity'] = signal.savgol_filter(df['Angular_Velocity'], window_size, polynomial_order)

    # Separar dataframes para velocidades y aceleraciones
    df_speed = df[['Time (sec)', 'Speed_X', 'Speed_Y', 'Speed']][df['Speed'] != 0].copy()
    df_acceleration = df[['Time (sec)', 'Acceleration_X', 'Acceleration_Y', 'Acceleration']][df['Acceleration'] != 0].copy()

    return df, df_speed, df_acceleration




def crear_dataframeRelativo(dfA, dfB):
    dfCombined = pd.merge(dfA, dfB, on='Time (sec)')
    dfCombined['Delta_Time'] = dfCombined['Time (sec)'].diff().fillna(0)

    # Posición relativa en X e Y
    dfCombined['Relative_Position_X'] = dfCombined['X_x'] - dfCombined['X_y']
    dfCombined['Relative_Position_Y'] = dfCombined['Y_x'] - dfCombined['Y_y']

    # Calcular coordenadas polares de la posición relativa
    dfCombined['R_Relative'], dfCombined['Theta_Relative'] = cartesian_to_polar(
        dfCombined['Relative_Position_X'], dfCombined['Relative_Position_Y']
    )

    # Diferencia de Theta entre el borde y el centro
    dfCombined['Relative_Theta'] = dfCombined['Theta_x'] - dfCombined['Theta_y']

    # Velocidad angular relativa
    dfCombined['Delta_Relative_Theta'] = dfCombined['Relative_Theta'].diff().fillna(0)
    dfCombined['Relative_Angular_Velocity'] = dfCombined['Delta_Relative_Theta'] / dfCombined['Delta_Time']
    dfCombined['Relative_Angular_Velocity'] = dfCombined['Relative_Angular_Velocity'].replace([np.inf, -np.inf], 0)

    # Calcular velocidades y aceleraciones relativas
    dfCombined['Delta_X'] = dfCombined['Relative_Position_X'].diff().fillna(0)
    dfCombined['Delta_Y'] = dfCombined['Relative_Position_Y'].diff().fillna(0)
    dfCombined['Relative_Speed_X'] = dfCombined['Delta_X'] / dfCombined['Delta_Time']
    dfCombined['Relative_Speed_Y'] = dfCombined['Delta_Y'] / dfCombined['Delta_Time']
    dfCombined['Relative_Speed_X'] = signal.savgol_filter(dfCombined['Relative_Speed_X'], 26, 2)
    dfCombined['Relative_Speed_Y'] = signal.savgol_filter(dfCombined['Relative_Speed_Y'], 26, 2)

    dfCombined['Relative_Acceleration_X'] = dfCombined['Relative_Speed_X'].diff() / dfCombined['Delta_Time']
    dfCombined['Relative_Acceleration_Y'] = dfCombined['Relative_Speed_Y'].diff() / dfCombined['Delta_Time']
    dfCombined['Relative_Acceleration_X'] = signal.savgol_filter(dfCombined['Relative_Acceleration_X'], 26, 2)
    dfCombined['Relative_Acceleration_Y'] = signal.savgol_filter(dfCombined['Relative_Acceleration_Y'], 26, 2)

    # Crear dataframes de velocidad y aceleración relativas
    dfCombined_speed = dfCombined[['Time (sec)', 'Relative_Speed_X', 'Relative_Speed_Y']]
    dfCombined_acceleration = dfCombined[['Time (sec)', 'Relative_Acceleration_X', 'Relative_Acceleration_Y']]

    return dfCombined, dfCombined_speed, dfCombined_acceleration


In [7]:
# Creamos los DataFrames para los puntos
dfCentro, dfCentro_speed, dfCentro_acceleration = crear_dataframe(positionsCentro_m)
dfBorde, dfBorde_speed, dfBorde_acceleration = crear_dataframe(positionsBorde_m)
dfRelativoBordeCentro, dfRelativoBordeCentro_speed, dfRelativoBordeCentro_acceleration = crear_dataframeRelativo(dfBorde, dfCentro)

# Guardamos los DataFrames actualizados en archivos CSV (es para chequear después si todo está en orden)
dfCentro.to_csv('DataFrameA.csv', index=False, float_format='%.6f')
dfBorde.to_csv('DataFrameB.csv', index=False, float_format='%.6f')
dfRelativoBordeCentro.to_csv('DataFrameC.csv', index=False, float_format='%.6f')
print("Datos de posiciones, velocidades y velocidad angular relativa guardados en archivos CSV.")

# Comprobación opcional de conteo de filas
#print(dfCentro_acceleration.count())
#print(dfCentro_speed.count())
#print(dfRelativoBordeCentro.count())

Datos de posiciones, velocidades y velocidad angular relativa guardados en archivos CSV.


# Graficos

In [8]:
pio.renderers.default = 'vscode'  # 'browser' | 'vscode' | colab

In [9]:
# Elegir dataframe a graficar
dfG = dfCentro
dfG_speed = dfCentro_speed[dfCentro_speed['Time (sec)'] < 0.4].copy() # PROVISIONAL
dfG_acceleration = dfCentro_acceleration[dfCentro_acceleration['Time (sec)'] < 0.4].copy() # PROVISIONAL

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Suponiendo que tienes tus DataFrames cargados (dfG, dfG_speed, dfG_acceleration)

# Crear figura y ejes
fig = make_subplots(rows=4, cols=2, subplot_titles=("Trayectoria del Objeto", "Velocidad del Objeto", "Posición X / Tiempo", "Posición Y / Tiempo","Velocidad X / Tiempo", "Velocidad Y / Tiempo", "Aceleración X / Tiempo", "Aceleración Y / Tiempo"))

# Grafica la trayectoria
fig.update_xaxes(title_text="Posición X (m)", range=[0.35, 0.6], row=1, col=1, scaleanchor="y") # Relación 1:1 entre X e Y
fig.update_yaxes(title_text="Posición Y (m)", row=1, col=1) 
fig.add_trace(go.Scatter(x=dfG['X'], y=dfG['Y'], mode='lines', marker=dict(color='blue')), row=1, col=1)

# Grafica la velocidad
fig.update_yaxes(title_text="Velocidad (m/s)", row=1, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=1, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed'], mode='lines', marker=dict(color='red'), name='Velocidad Total'), row=1, col=2)

# Grafica la posición X / tiempo
fig.update_yaxes(title_text="Posición X (m)", range=[0.35,0.6], row=2, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['X'], mode='lines', marker=dict(color='green')), row=2, col=1)

# Grafica la posición Y / tiempo
fig.update_yaxes(title_text="Posición Y (m)", row=2, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=2)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Y'], mode='lines', marker=dict(color='magenta')), row=2, col=2)

# Grafica la velocidad X / tiempo
fig.update_yaxes(title_text="Velocidad X (m/s)", range=[-0.5,0.5], row=3, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=1)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_X'], mode='lines', marker=dict(color='blue')), row=3, col=1)

# Grafica la velocidad Y / tiempo
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=3, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_Y'], mode='lines', marker=dict(color='red')), row=3, col=2)

# Grafica la aceleración X / tiempo
fig.update_yaxes(title_text="Aceleración X (m/s^2)", row=4, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=1)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_X'], mode='lines', marker=dict(color='green')), row=4, col=1)

# Grafica la aceleración Y / tiempo
fig.update_yaxes(title_text="Aceleración Y (m/s^2)", row=4, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=2)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_Y'], mode='lines', marker=dict(color='purple')), row=4, col=2)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1000, width=1200, title_text="Análisis de Movimiento del Centro del Objeto", showlegend=False)

# Mostrar los gráficos
fig.show()

In [10]:
# Elegir dataframe a graficar
dfG = dfBorde
dfG_speed = dfBorde_speed[dfBorde_speed['Time (sec)'] < 0.4].copy()
dfG_acceleration = dfBorde_acceleration[dfBorde_acceleration['Time (sec)'] < 0.4].copy()

# Crear figura y ejes
# 5 filas, 2 columnas
fig = make_subplots(
    rows=5, cols=2,
    subplot_titles=(
        "Trayectoria del Objeto", "Velocidad del Objeto",
        "Posición X / Tiempo", "Posición Y / Tiempo",
        "Velocidad X / Tiempo", "Velocidad Y / Tiempo",
        "Aceleración X / Tiempo", "Aceleración Y / Tiempo",
        "Velocidad Angular / Tiempo", ""
    )
)

# Grafica la trayectoria
fig.update_yaxes(title_text="Posición Y (m)", row=1, col=1)
fig.update_xaxes(title_text="Posición X (m)", row=1, col=1, scaleanchor="y")
fig.add_trace(go.Scatter(x=dfG['X'], y=dfG['Y'], mode='lines', marker=dict(color='blue')), row=1, col=1)

# Grafica la velocidad
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=1, col=2)
fig.update_xaxes(title_text="Velocidad X (m/s)", row=1, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed'], mode='lines', marker=dict(color='red'), name='Velocidad Total'), row=1, col=2)

# Grafica la posición X en función del tiempo
fig.update_yaxes(title_text="Posición X (m)", row=2, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['X'], mode='lines', marker=dict(color='green')), row=2, col=1)

# Grafica la posición Y / tiempo
fig.update_yaxes(title_text="Posición Y (m)", row=2, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=2)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Y'], mode='lines', marker=dict(color='magenta')), row=2, col=2)

# Grafica la velocidad X / tiempo
fig.update_yaxes(title_text="Velocidad X (m/s)", row=3, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=1)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_X'], mode='lines', marker=dict(color='blue')), row=3, col=1)

# Grafica la velocidad Y / tiempo
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=3, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_Y'], mode='lines', marker=dict(color='red')), row=3, col=2)

# Grafica la aceleración X / tiempo
fig.update_yaxes(title_text="Aceleración X (m/s^2)", row=4, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=1)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_X'], mode='lines', marker=dict(color='green')), row=4, col=1)

# Grafica la aceleración Y / tiempo
fig.update_yaxes(title_text="Aceleración Y (m/s^2)", row=4, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=2)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_Y'], mode='lines', marker=dict(color='purple')), row=4, col=2)

# Grafica la velocidad angular
fig.update_yaxes(title_text="Velocidad Angular (rad/s)", row=5, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=5, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Angular_Velocity'], mode='lines', marker=dict(color='orange')), row=5, col=1)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1200, width=1200, title_text="Análisis de Movimiento del Borde del Objeto", showlegend=False)

# Mostrar los gráficos
fig.show()

In [11]:
# Elegir dataframe a graficar
dfG = dfRelativoBordeCentro
dfG_speed = dfRelativoBordeCentro_speed
dfG_acceleration = dfRelativoBordeCentro_acceleration

# Crear figura y ejes
# 3 filas, 2 columnas
fig = make_subplots(rows=4, cols=2, subplot_titles=("Posición X / Tiempo", "Posición Y / Tiempo",
                                                    "Velocidad X / Tiempo", "Velocidad Y / Tiempo", 
                                                    "Aceleración X / Tiempo", "Aceleración Y / Tiempo",
                                                    "Velocidad Angular / Tiempo"))

# Grafica la posición X en funcion del tiempo
fig.update_yaxes(title_text="Posición X (m)", row=1, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=1, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Position_X'], mode='lines', marker=dict(color='green')), row=1, col=1)

# Grafica la posición Y / tiempo
fig.update_yaxes(title_text="Posición Y (m)", row=1, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=1, col=2)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Position_Y'], mode='lines', marker=dict(color='magenta')), row=1, col=2)

# Grafica la velocidad X / tiempo
fig.update_yaxes(title_text="Velocidad X (m/s)", row=2, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=1)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Relative_Speed_X'], mode='lines', marker=dict(color='blue')), row=2, col=1)

# Grafica la velocidad Y / tiempo
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=2, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Relative_Speed_Y'], mode='lines', marker=dict(color='red')), row=2, col=2)

# Grafica la aceleración X / tiempo
fig.update_yaxes(title_text="Aceleración X (m/s^2)", row=3, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=1)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Relative_Acceleration_X'], mode='lines', marker=dict(color='green')), row=3, col=1)

# Grafica la aceleración Y / tiempo
fig.update_yaxes(title_text="Aceleración Y (m/s^2)", row=3, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=2)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Relative_Acceleration_Y'], mode='lines', marker=dict(color='purple')), row=3, col=2)

# Grafica la velocidad angular
fig.update_yaxes(title_text="Velocidad Angular (rad/s)", row=4, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Angular_Velocity'], mode='lines', marker=dict(color='orange')), row=4, col=1)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1000, width=1200, title_text="Análisis del movimiento relativo de ambos puntos", showlegend=False)

# Mostrar los gráficos
fig.show() # Si hay error acá, revisar el renderer

KeyError: 'Angular_Velocity_Relative'